<a href="https://colab.research.google.com/github/alonzojp/Magnolia-Beach-Model/blob/main/Functionized_Magnolia_Beach_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Imports
import pandas as pd
import plotly.graph_objects as go
import math
import plotly.express as px
import tensorflow as tf
import random

import warnings
warnings.filterwarnings("ignore")

### **Ports Dataframe**

In [2]:
# Combine Port Lavaca & OConnor data together
def generate_ports_df(LAVACA_DATA_PATH, OCONNOR_DATA_PATH):
  lavaca_df = pd.read_csv(LAVACA_DATA_PATH)[:-11]
  oconnor_df = pd.read_csv(OCONNOR_DATA_PATH)[:-11]
  ports_df = pd.concat([lavaca_df, oconnor_df], axis=1)

  ports_df.index = lavaca_df['#date+time']
  ports_df.drop(columns=['#date+time'], inplace=True)
  ports_df.index = pd.to_datetime(ports_df.index, format='%m-%d-%Y %H:%M')

  ports_df[ports_df.columns] = ports_df[ports_df.columns].apply(pd.to_numeric, errors='coerce')

  ports_df = ports_df.drop(columns = ['057-wsd', '057-wdr'])
  ports_df.drop(ports_df['2016-10-30':'2016-11-08'].index, inplace=True) # 033-wsd error

  return ports_df

In [3]:
# Visualization of all data in dataset
def graph_data(df):
  for column in df.columns:
    fig = go.Figure(data=go.Scatter(x=df.index, y=df[column], mode='lines', line_color='#003f5c', name = ""))
    fig.update_traces(connectgaps=False)
    fig.update_layout(title = column, xaxis_title="", yaxis_title="")
    fig.show()

In [4]:
# Converts wsd and wdr to X and Y wind data
def convert_winds(ports_df):
  ports = ['033']
  for port in ports:
    x_wind = []
    y_wind = []
    for i in range(0, len(ports_df)):
      radius = ports_df[port + '-wsd'][i]
      theta = ports_df[port + '-wdr'][i]
      theta = theta * (math.pi / 180)
      x_wind.append(radius * math.cos(theta))
      y_wind.append(radius * math.sin(theta))
    ports_df[port + "-Xwind"] = x_wind
    ports_df[port + "-Ywind"] = y_wind
    del ports_df[port + '-wsd']
    del ports_df[port + '-wdr']
  return ports_df

In [5]:
# Creates estimated Magnolia data
def create_magnolia_data(ports_df):
  ports_df['mag-pwl'] = (ports_df['033-pwl'] + ports_df['057-pwl']) / 2
  ports_df['mag-surge'] = (ports_df['033-surge'] + ports_df['057-surge']) / 2
  ports_df['mag-harmwl'] = (ports_df['033-harmwl'] + ports_df['057-harmwl']) / 2
  return ports_df

In [6]:
LAVACA_DATA_PATH = '/content/Lavaca Data 09_24.csv' #033
OCONNOR_DATA_PATH = '/content/OConnor Data 09_24.csv' #057

ports_df = generate_ports_df(LAVACA_DATA_PATH, OCONNOR_DATA_PATH)
ports_df = create_magnolia_data(ports_df)
ports_df = convert_winds(ports_df)
# graph_data(ports_df)
ports_df

,033-pwl,033-harmwl,033-surge,057-pwl,057-harmwl,057-surge,mag-pwl,mag-surge,mag-harmwl,033-Xwind,033-Ywind
#date+time,,,,,,,,,,,
2010-01-01 00:00:00,0.070,-0.149,0.219,0.153,0.032,0.121,0.1115,0.1700,-0.0585,NaN,NaN
2010-01-01 01:00:00,0.124,-0.083,0.207,0.191,0.073,0.118,0.1575,0.1625,-0.0050,NaN,NaN
2010-01-01 02:00:00,0.182,-0.017,0.199,0.216,0.108,0.108,0.1990,0.1535,0.0455,NaN,NaN
2010-01-01 03:00:00,0.224,0.042,0.182,0.292,0.133,0.159,0.2580,0.1705,0.0875,NaN,NaN
2010-01-01 04:00:00,0.202,0.093,0.109,0.267,0.148,0.119,0.2345,0.1140,0.1205,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2025-01-01 19:00:00,NaN,-0.231,NaN,NaN,-0.207,NaN,NaN,NaN,-0.2190,NaN,NaN
2025-01-01 20:00:00,NaN,-0.272,NaN,NaN,-0.195,NaN,NaN,NaN,-0.2335,NaN,NaN
2025-01-01 21:00:00,NaN,-0.295,NaN,NaN,-0.169,NaN,NaN,NaN,-0.2320,NaN,NaN


### **ANN #1**

In [7]:
# Generates first ANN dataframe
def create_first_ann_df(ports_df, LEAD_TIME):
  ports_df['TARGET'] = ports_df['mag-surge'].shift(-1 * LEAD_TIME + 5) # accounts for 5hr delay
  ports_df['REMOVE (for later conversion)'] = ports_df['mag-harmwl'].shift(-1 * LEAD_TIME + 5) # used later for conversion
  ports_df = ports_df[['033-pwl',	'033-surge',	'033-Xwind',	'033-Ywind', '057-pwl',	'057-surge', 'mag-pwl', 'mag-surge', 'TARGET', 'REMOVE (for later conversion)']]

  organized_df = pd.DataFrame() # adds 'hours back' features
  column_names = ['033-pwl',	'033-surge',	'057-pwl',	'057-surge', 'mag-pwl', 'mag-surge', '033-Xwind',	'033-Ywind']
  for column in column_names:
    for i in range(24, 0, -1):
      df_shifted = ports_df.shift(1 * i)
      organized_df[column + " -" + str(i)] = df_shifted[column]
    organized_df[column] = ports_df[column]

  first_winds_df = pd.DataFrame() # first ANN's wind perfect prog
  for i in range(1, LEAD_TIME + 1 - 5):
    df_reverse_shifted = ports_df.shift(-1 * i)
    first_winds_df['033-Xwind' + " +" + str(i)] = df_reverse_shifted['033-Xwind']
  for i in range(1, LEAD_TIME + 1 - 5):
    df_reverse_shifted = ports_df.shift(-1 * i)
    first_winds_df['033-Ywind' + " +" + str(i)] = df_reverse_shifted['033-Ywind']

  second_winds_df = pd.DataFrame() # second ANN's wind perfect prog
  for i in range(1, LEAD_TIME + 1):
    df_reverse_shifted = ports_df.shift(-1 * i)
    second_winds_df['033-Xwind' + " +" + str(i)] = df_reverse_shifted['033-Xwind']
  for i in range(1, LEAD_TIME + 1):
    df_reverse_shifted = ports_df.shift(-1 * i)
    second_winds_df['033-Ywind' + " +" + str(i)] = df_reverse_shifted['033-Ywind']

  first_df = pd.concat([organized_df, first_winds_df], axis=1) # creating first dataset
  first_df['TARGET'] = ports_df['TARGET']
  first_df['REMOVE (for later conversion)'] = ports_df['REMOVE (for later conversion)']
  first_df = first_df.dropna()
  first_pwl_conversion = first_df['REMOVE (for later conversion)']
  del first_df['REMOVE (for later conversion)']
  return first_df, first_pwl_conversion, second_winds_df

In [8]:
# Generates first ANN data split
def generate_first_ann_split(first_df):
  length = len(first_df)
  train_length = round(length * .6)
  val_length = round(length * .8)

  train_df = first_df[:train_length]
  val_df = first_df[train_length:val_length]
  test_df = first_df[val_length:]

  X_train = train_df.drop('TARGET',axis=1)
  y_train = train_df['TARGET']
  X_val = val_df.drop('TARGET',axis=1)
  y_val = val_df['TARGET']
  X_test = test_df.drop('TARGET',axis=1)
  y_test = test_df['TARGET']

  return X_train, y_train, X_val, y_val, X_test, y_test

In [9]:
# Plots train, val, and test distribution
def plot_first_ann(y_train, y_val, y_test):
  fig_train = px.line(y_train)
  fig_train.update_traces(line_color='#EF2648')
  fig_train.update_traces(name='Training')

  fig_val = px.line(y_val)
  fig_val.update_traces(line_color='#FBC800')
  fig_val.update_traces(name='Validation')

  fig_test = px.line(y_test)
  fig_test.update_traces(line_color='#5A8A91')
  fig_test.update_traces(name='Testing')

  fig = go.Figure(data = fig_train.data + fig_val.data + fig_test.data)
  fig.show()

In [10]:
# Trains first ANN
def train_first_ann(X_train, y_train, X_val, y_val, X_test, y_test, units1, activation1):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                  patience=20,
                                                  mode='min')

  model = tf.keras.Sequential([
      tf.keras.layers.Dense(units1,
                            activation= activation1,
                            input_shape = [X_train.shape[1]],
                            kernel_regularizer = tf.keras.regularizers.L2(l2=0.0001)
                            ),
      tf.keras.layers.Dense(1),
  ])

  model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])

  losses = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    callbacks=[early_stopping],
                    epochs=1000, batch_size = 32,
                    shuffle=True, verbose = 0)

  return model, losses

In [11]:
# Plots training loss curve
def plot_loss_curve(losses):
  loss_df = pd.DataFrame(losses.history)
  loss_df.loc[:,['loss','val_loss']].plot()

In [12]:
# Generate first ANN predictions
def generate_first_ann_predictions(model, X_test, y_test):
  predictions = model.predict(X_test, verbose = 0)

  predictions_df = pd.DataFrame()
  predictions_df.index = X_test.index
  predictions_df['Predictions'] = predictions
  predictions_df['Actual'] = y_test

  predictions_df = predictions_df.resample('60min').mean()
  return predictions_df

In [13]:
# Converts predictted surge to primary water level
def convert_surge_to_pwl(predictions_df, pwl_conversion):
  filtered_conversion_df = pwl_conversion[predictions_df.index[0]:predictions_df.index[-1]]

  predictions_df['PredictedPWL'] = predictions_df['Predictions'] + filtered_conversion_df
  predictions_df['ActualPWL'] = predictions_df['Actual'] + filtered_conversion_df
  return predictions_df

In [14]:
# Plot predictions against actual for 1st ANN
def plot_first_ann_predictions(predictions_df):
  fig = go.Figure(data=go.Scatter(x=predictions_df.index, y=predictions_df['ActualPWL'], mode='lines', line_color='#000000', name = "Actual"))
  fig.add_trace(go.Scatter(x=predictions_df.index, y=predictions_df['PredictedPWL'],mode='lines', line=dict(color="#FF0000"), name = "Predictions"))
  fig.update_traces(connectgaps=False)
  predictions_df = predictions_df.dropna()
  fig.show()

In [15]:
# Produces evaluations for the first ANN
def evaluate_first_ann(predictions_df):
  temp = predictions_df.copy().dropna()
  mae = sum(abs(temp['PredictedPWL'] - temp['ActualPWL'])) / len(temp['PredictedPWL'])
  mae = round(mae * 100, 2)

  counter = 0
  array = abs(temp['PredictedPWL'] - temp['ActualPWL'])
  # array = np.array(array)
  for i in range(0, len(array)):
    if(array[i] > .15):
      counter += 1

  cf = (float(f'{100- (counter / len(array) * 100):.2f}'))
  return mae, cf

### **Magnolia Beach**

In [16]:
# Creates magnolia dataframe with correct timestamps
def create_magnolia_df(MAGNOLIA_DATA_PATH, CHANGING_TIME):
  magnolia_df = pd.read_csv(MAGNOLIA_DATA_PATH, index_col = 0)
  magnolia_df = magnolia_df.dropna() # remove data inconsistency
  magnolia_df.index = pd.to_datetime(magnolia_df.index, format='mixed', utc=True) # Converting index to DateTimeIndex
  magnolia_df.index = pd.to_datetime(magnolia_df.index, format='%Y-%m-%d %H:%M') # converts index to date
  magnolia_df.index = magnolia_df.index.tz_localize(None)
  magnolia_df.index.names = ['Timestamp (UTC)']
  magnolia_df['Distance to Water Measurement (m)'] = magnolia_df['Distance to Water Measurement (mm)'] / 1000
  del magnolia_df['Distance to Water Measurement (mm)']
  magnolia_df = magnolia_df.resample('60min').mean()

  magnolia_df['TARGET (+' + str(CHANGING_TIME) + ')'] = magnolia_df['Distance to Water Measurement (m)'].shift(-1 * CHANGING_TIME)
  magnolia_df = magnolia_df.dropna()

  return magnolia_df

### **ANN #2**

In [17]:
def create_second_ann_df(predictions_df, magnolia_df, second_winds_df, CHANGING_TIME):
  second_df = predictions_df[['PredictedPWL']].resample('60min').mean()
  second_df = second_df[magnolia_df.index[0]:magnolia_df.index[-1]]
  second_df['TARGET (+' + str(CHANGING_TIME) + ')'] = magnolia_df['TARGET (+' + str(CHANGING_TIME) + ')']
  second_df = second_df.rename(columns={'PredictedPWL': 'PredictedPWL (+' + str(CHANGING_TIME - 5) + ')'})
  second_df = second_df.dropna()

  organized_df = pd.DataFrame()
  column_names = ['PredictedPWL (+' + str(CHANGING_TIME - 5) + ')']

  for column in column_names:
    for i in range(12, 0, -1):
      if(i % 1 == 0): # Interchangeable
        df_shifted = second_df.shift(1 * i)
        organized_df[column + " -" + str(i)] = df_shifted[column]
    organized_df[column] = second_df[column]


  temp1 = organized_df.dropna().resample('60min').mean()
  temp2 = second_winds_df[organized_df.index[0]:organized_df.index[-1]].resample('60min').mean()
  final_second_df = temp1.set_index(temp1.index).join(temp2.set_index(temp2.index))

  final_second_df['TARGET (+' + str(CHANGING_TIME)  + ')'] = magnolia_df['TARGET (+' + str(CHANGING_TIME)  + ')']
  final_second_df = final_second_df.dropna()

  return final_second_df

### **Main Loop**

In [18]:
# all of this will be in a for loop eventually, with changing_time iterating through different lead times
CHANGING_TIME = 24
random_id = random.randint(0, 10000)


# FIRST ANN
# ----------------------------------------------------------------------------------------------------
first_df, first_pwl_conversion, second_winds_df = create_first_ann_df(ports_df, CHANGING_TIME)
first_X_train, first_y_train, first_X_val, first_y_val, first_X_test, first_y_test = generate_first_ann_split(first_df)
# plot_first_ann(first_y_train, first_y_val, first_y_test)

first_model, first_losses = train_first_ann(first_X_train, first_y_train, first_X_val, first_y_val, first_X_test, first_y_test, 256, 'relu')
first_predictions_df = generate_first_ann_predictions(first_model, first_X_test, first_y_test)
converted_first_predictions_df = convert_surge_to_pwl(first_predictions_df, first_pwl_conversion)
# plot_first_ann_predictions(converted_first_predictions_df)

first_mae, first_cf = evaluate_first_ann(converted_first_predictions_df)

first_model_name = f'{random_id}_1stANN_{CHANGING_TIME}hr_{first_mae}mae_{first_cf}cf15.keras'
first_model.save(first_model_name)
# ----------------------------------------------------------------------------------------------------


# MAGNOLIA
# ----------------------------------------------------------------------------------------------------
MAGNOLIA_DATA_PATH = '/content/Magnolia Data 09_24.csv' #mag
magnolia_df = create_magnolia_df(MAGNOLIA_DATA_PATH, CHANGING_TIME)[:-100] # to account for not enough data
# graph_data(magnolia_df)
# ----------------------------------------------------------------------------------------------------


# SECOND ANN
# ----------------------------------------------------------------------------------------------------
second_df = create_second_ann_df(converted_first_predictions_df, magnolia_df, second_winds_df, CHANGING_TIME)
second_df
# ----------------------------------------------------------------------------------------------------

,PredictedPWL (+19) -12,PredictedPWL (+19) -11,PredictedPWL (+19) -10,PredictedPWL (+19) -9,PredictedPWL (+19) -8,PredictedPWL (+19) -7,PredictedPWL (+19) -6,PredictedPWL (+19) -5,PredictedPWL (+19) -4,PredictedPWL (+19) -3,...,033-Ywind +16,033-Ywind +17,033-Ywind +18,033-Ywind +19,033-Ywind +20,033-Ywind +21,033-Ywind +22,033-Ywind +23,033-Ywind +24,TARGET (+24)
#date+time,,,,,,,,,,,,,,,,,,,,,
2023-06-13 04:00:00,0.319344,0.322142,0.314953,0.306566,0.289666,0.290402,0.298925,0.305186,0.308939,0.306573,...,1.889589,3.146695,3.332822,1.959567,2.701959,3.046128,2.852337,2.651923,1.862799,2.130800
2023-06-13 05:00:00,0.322142,0.314953,0.306566,0.289666,0.290402,0.298925,0.305186,0.308939,0.306573,0.295988,...,3.146695,3.332822,1.959567,2.701959,3.046128,2.852337,2.651923,1.862799,1.822104,2.143748
2023-06-13 06:00:00,0.314953,0.306566,0.289666,0.290402,0.298925,0.305186,0.308939,0.306573,0.295988,0.282552,...,3.332822,1.959567,2.701959,3.046128,2.852337,2.651923,1.862799,1.822104,-0.423921,2.148842
2023-06-13 07:00:00,0.306566,0.289666,0.290402,0.298925,0.305186,0.308939,0.306573,0.295988,0.282552,0.262350,...,1.959567,2.701959,3.046128,2.852337,2.651923,1.862799,1.822104,-0.423921,-1.221178,2.147310
2023-06-13 08:00:00,0.289666,0.290402,0.298925,0.305186,0.308939,0.306573,0.295988,0.282552,0.262350,0.234958,...,2.701959,3.046128,2.852337,2.651923,1.862799,1.822104,-0.423921,-1.221178,-1.024524,2.146400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-24 17:00:00,0.236043,0.236331,0.249833,0.276622,0.316401,0.358409,0.411391,0.458841,0.492784,0.526034,...,-1.090272,-0.771345,-1.104606,-0.280299,-0.115139,-0.539883,0.453660,0.451485,0.920505,1.867450
2024-09-24 18:00:00,0.236331,0.249833,0.276622,0.316401,0.358409,0.411391,0.458841,0.492784,0.526034,0.546804,...,-0.771345,-1.104606,-0.280299,-0.115139,-0.539883,0.453660,0.451485,0.920505,2.772751,1.882600
2024-09-24 19:00:00,0.249833,0.276622,0.316401,0.358409,0.411391,0.458841,0.492784,0.526034,0.546804,0.550709,...,-1.104606,-0.280299,-0.115139,-0.539883,0.453660,0.451485,0.920505,2.772751,2.404209,1.901500
